In [193]:
import os, json
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [195]:
#can use export to set environment variable.(temporary way)
#can set environment variable in zsh(is a global approach)
#create a .env file and fetch it from there(for project scope)

#once we have created a .env file then we would have to load the env file.
#we can use python dotenv package to load the .env file

load_dotenv()


True

In [196]:
KEY = os.getenv('OPENAI_API_KEY')

In [198]:
def key_value_pair(argument):
    d = {}
    key = argument.split("=")[0]
    value = argument.split('=')[1]
    d[key] = value
    return d

In [199]:
args = key_value_pair(KEY)


In [200]:
llm = ChatOpenAI(openai_api_key=args['OPENAI_API_KEY'],model_name='gpt-3.5-turbo',temperature=0.5)

In [202]:
response_json = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",

    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",

    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",

    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",

    }
}
template = """
Text:{input_text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format response like response_json below and use it as a guide. \
Ensure to make {number} MCQs
### response_json
{response_json} 

"""

#['text','number','subjext','tone','response_json'] are the variable which user will be passing
mcq_generation_prompt = PromptTemplate(
    input_variables=['input_text','number','subject','tone','response_json'],
    template=template
)

In [206]:
#now we would be using llm chains to connect several components together..
#in this we would be connecting our llm and prompt together

quiz_chain = LLMChain(llm=llm,
                      prompt=mcq_generation_prompt,
                      output_key="quiz",
                      verbose=True)

In [207]:
template2 = """
You are an expert english gramarian and writer. Given a Multiple choice quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use max of 50 words for complexity.
If the quiz is not at per with cognitive and analytical abilities of the students.\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits with student abilities.
quic mcqs:
{quiz}
"""
quiz_evaluation_prompt = PromptTemplate(input_variables=['subject','quiz'],template=template2)

In [209]:
review_chain = LLMChain(llm = llm,
                        prompt = quiz_evaluation_prompt,
                        output_key="reviews",
                        
                        verbose=True)

In [210]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain,review_chain],
                                          input_variables = ['input_text','number','subject','tone','response_json'],
                                          output_variables = ['quiz','reviews'],
                                          verbose = True,)

In [212]:
file_path = '/Users/kushagra/gen_ai/mcq_generator/data.txt'
with open (file_path,'r') as file:
    text = file.read()



In [214]:
number = 5
subject = "machine learning"
tone = "simple"

In [215]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'input_text':text,
            'number':number,
            'subject':subject,
            'tone':tone,
            'response_json':json.dumps(response_json)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [218]:
print(f"total_tokens: {cb.total_tokens}")
print(f"prompt_token:{cb.prompt_tokens}")
print(f"completion_tokens:{cb.completion_tokens}")
print(f"total_cost:{cb.total_cost}")

total_tokens: 1808
prompt_token:1398
completion_tokens:410
total_cost:0.002917


In [220]:
response

{'input_text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive s

In [225]:
quiz = json.loads(response['quiz'])

In [253]:
# data prep for creating dataframe 
quiz_table_data = []
for key,value in quiz.items():
    temp_dict = {}
    temp_dict['MCQ'] = value['mcq']
    choices = value['options']
    choices_str = ''
    for choices_key,choices_value in choices.items():
        choices_str = choices_str + choices_key +': ' + choices_value + ' |' + ' '
    temp_dict['Choices'] = choices_str
    temp_dict['Correct Answer'] = value['correct']
    quiz_table_data.append(temp_dict)

quiz_table_data


[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Alan Turing | b: Donald Hebb | c: Arthur Samuel | d: Tom M. Mitchell | ',
  'Correct Answer': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cyberdyne | b: Cybernet | c: Cybertron | d: Cyberspace | ',
  'Correct Answer': 'c'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks that inspired machine learning algorithms?',
  'Choices': 'a: Walter Pitts | b: Warren McCulloch | c: Donald Hebb | d: Raytheon Company | ',
  'Correct Answer': 'a'},
 {'MCQ': 'What is the main purpose of modern-day machine learning algorithms?',
  'Choices': 'a: To confuse data | b: To make predictions for future outcomes | c: To hide patterns | d: To complicate tasks | ',
  'Correct Answer': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of a computer program learning from experience?',
  'Choices': 'a

In [256]:
#creating dataframe
quiz = pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct Answer
0,Who coined the term machine learning in 1959?,a: Alan Turing | b: Donald Hebb | c: Arthur Sa...,c
1,What was the name of the experimental 'learnin...,a: Cyberdyne | b: Cybernet | c: Cybertron | d:...,c
2,Who proposed the early mathematical models of ...,a: Walter Pitts | b: Warren McCulloch | c: Don...,a
3,What is the main purpose of modern-day machine...,a: To confuse data | b: To make predictions fo...,b
4,"According to Tom M. Mitchell, what is the defi...",a: Its performance at tasks in T improves with...,a


In [257]:
quiz.to_csv("machinelearning.csv",index=False)